<a href="https://colab.research.google.com/github/molinamarriott/challenges/blob/main/WebScrappingSky.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# install chromium, its driver, and selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium
!pip install webdriver-manager

Hit:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:5 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:6 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:7 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:9 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Ign:10 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:12 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:13 http://security.ubuntu.com/ubuntu 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [31]:
import requests
from bs4 import BeautifulSoup

urls = ['https://www.skysports.com/premier-league-results/2022-23',
        'https://www.skysports.com/premier-league-results/2021-22',
        "https://www.skysports.com/premier-league-results/2020-21",
        "https://www.skysports.com/premier-league-results/2019-20"]

links = []

for url in urls:
  response = requests.get(url)
  soup = BeautifulSoup(response.text, 'html.parser')

  matches = soup.find_all('div', {'class': 'fixres__item'})

  for match in matches:
    link_match = str(match.find('a')["href"])
    links.append(link_match)

In [32]:
def visit_url(url):
  import requests
  from bs4 import BeautifulSoup
  from selenium import webdriver
  import pandas as pd


  chrome_options = webdriver.ChromeOptions()
  chrome_options.add_argument('--headless')
  chrome_options.add_argument('--no-sandbox')
  chrome_options.add_argument('--disable-dev-shm-usage')
  driver = webdriver.Chrome('chromedriver',options=chrome_options)
# Navigate to the page
  driver.get(url)

# Wait for the page to load
  driver.implicitly_wait(1000)

# Use BeautifulSoup to parse the page source
  html = driver.page_source
  soup = BeautifulSoup(html, 'html.parser')
  return soup

In [33]:
def get_variables(url_bruto):
  url_bruto[:-6] + 'teams/' + url_bruto[-6:]
  # Create a new instance of the Chrome driver
  url = url_bruto[:-6] + 'stats/' + url_bruto[-6:]

  soup = visit_url(url)

  table = soup.find("div",{"class":"sdc-site-match-stats__inner"})

  stats = table.find_all("h5", {'class':"sdc-site-match-stats__name"})
  
  variable_names = ["Home","Away"]
  values = []
  
  local = table.find("span", {'class':"sdc-site-match-stats__team-name"}).get_text(strip=True)
  away = table.find("span", {'class':"sdc-site-match-stats__team-name sdc-site-match-stats__team-name--away"}).get_text(strip=True)

  values.append(local)
  values.append(away)

  for stat in stats:
    variable_names.append(f'Home {stat.get_text(strip=True)}')
    variable_names.append(f'Away {stat.get_text(strip=True)}')

  results = soup.find_all("span", {'class':"sdc-site-match-stats__val"})
  for result in results:
    values.append(result.get_text(strip=True))

  fecha = soup.find("time").get_text(strip=True)
  variable_names.append("Fecha")
  values.append(fecha)

  url2 = url.replace("stats", "teams")
  soup = visit_url(url2)

  referee = soup.find("dd", {"class":'sdc-site-team-lineup__officials-name', 'data-officials-role':"Referee"}).get_text(strip=True)
  variable_names.append("Referee")
  values.append(referee)

  df = pd.DataFrame([values], columns=variable_names)

  return df

In [ ]:
n = 1
for link in links:
  if n == 1:
    df = get_variables(link)
  else:
    df = df.append(get_variables(link))
  n += 1

df